In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
dados=pd.read_csv("DadosDeCancer.csv")

In [3]:
dados2=pd.DataFrame(np.copy(dados),columns=dados.columns)

In [4]:
dados2.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,no


# A única variável que é numérica está com o seu tipo alterado pois veio de um array numpy, irei consertar em seguida.

In [5]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null object
breast        286 non-null object
irradiat      286 non-null object
dtypes: object(9)
memory usage: 20.2+ KB


In [6]:
#Ajeitando o tipo da variável deg-malig
dados2["deg-malig"]=np.array([int(i) for i in dados2["deg-malig"]])

In [7]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null int32
breast        286 non-null object
irradiat      286 non-null object
dtypes: int32(1), object(8)
memory usage: 19.1+ KB


# Mudando as classes para 0 e 1.

In [8]:
dados2["Classe"]=np.array([1 if i=="no-recurrence-events" else 0 for i in dados2["class"]])

# Usando a transformação OneHotEncoder para mudar as variáveis categóricas em numéricas, se baseia que em cada tipo único de cada variável fazer uma coluna de zeros e uns, 1 se o tipo único acontecer ou 0 caso contrário.

In [9]:
transformacao=OneHotEncoder(sparse=False)

# Organizando os dados necessários para o Perceptron, em que a primeira coluna é o peso do bias.

In [10]:
dados2=np.column_stack((np.full(dados2.shape[0],-1),transformacao
.fit_transform(dados2.loc[:,["age","menopause","tumor-size","inv-nodes"
,"node-caps","breast","irradiat"]]),dados2["deg-malig"],dados2["Classe"]))

In [11]:
x=dados2[:,:-1]
y=dados2[:,-1]

In [25]:
class MultiLayerPerceptronBackPropagation():
    def __init__(self,epocas,alpha,neuronios_por_camada):
        #Número de épocas
        self.epocas=epocas
        #Taxa de aprendizagem
        self.alpha=alpha
        #Array da quantidade de neuronios por camada
        self.neuronios_por_camada=neuronios_por_camada
    
    def fit(self,xtreino,ytreino):
        quantidade_de_pesos=np.concatenate((xtreino.shape[1],self.neuronios_por_camada)
        ,axis=None)  
        #Inicialização dos pesos aleatoriamente
        self.pesos=np.array([np.random.normal(size=(i,j)) for i,j in np.column_stack((
                            quantidade_de_pesos[:-1],quantidade_de_pesos[1:]))])
        #Pondo o bias corretamente em todas camada oculta.
        self.pesos=np.array([np.row_stack((self.pesos[i],np.repeat(
        -np.random.normal(size=1),repeats=self.pesos[i].shape[1])))
        if i!=0 else self.pesos[i] for i in range(self.pesos.shape[0])])
        for e in range(self.epocas):
            previsoes=[]
            for j in range(xtreino.shape[0]):
                            for i in range(self.neuronios_por_camada.shape[0]):
                                if i==0:
                                    #Camada de entrada para a camada oculta
                                    entrada=np.concatenate((np.array([xtreino[j,:]
                                    .dot(self.pesos[i])]),1),axis=None)
                                    entrada2=MultiLayerPerceptronBackPropagation(self.epocas,self.alpha
                                                            ,self.neuronios_por_camada).sigmoide(entrada)
                                else: 
                                    #Camada oculta para a saída
                                    ycalculado=entrada2.dot(self.pesos[i])
                                    yestimado=MultiLayerPerceptronBackPropagation(self.epocas,self.alpha
                                                                ,self.neuronios_por_camada).sigmoide(ycalculado)
                                    previsoes.append(np.round(yestimado)[0])
                            #Atualização dos pesos
                            for w in range(len(self.pesos[1])):
                                self.pesos[1][w]=self.pesos[1][w]+self.alpha*entrada2[w]*((ytreino[j]
                                -MultiLayerPerceptronBackPropagation(self.epocas,self.alpha,self.neuronios_por_camada)
                                .sigmoide(ycalculado))*MultiLayerPerceptronBackPropagation(self.epocas,self.alpha
                                                ,self.neuronios_por_camada).dsigmoide(ycalculado))
                            for a in range(self.pesos[0].shape[0]):
                                for b in range(self.pesos[0].shape[1]):
                                    self.pesos[0][a,b]=(self.pesos[0][a,b]+
                            self.alpha*MultiLayerPerceptronBackPropagation(self.epocas,self.alpha
                            ,self.neuronios_por_camada).dsigmoide(entrada[b])
                            *((ytreino[j]-yestimado)*MultiLayerPerceptronBackPropagation(self.epocas,self.alpha
                            ,self.neuronios_por_camada).dsigmoide(ycalculado)*self.pesos[1][b]*xtreino[j,a]))       
            print("Função custo é: ",np.sum((ytreino-previsoes)**2),"na",e+1,"época")
            
    def predict(self,xteste):
        #Com os pesos já passados pelo backpropagaion, agora só resta a previsões dos dados de teste.
        previsoes=[]    
        for j in range(xteste.shape[0]):   
            for i in range(self.neuronios_por_camada.shape[0]):
                if i==0:
                    entrada=np.concatenate((np.array([xteste[j,:]
                    .dot(self.pesos[i])]),1),axis=None)
                    entrada2=MultiLayerPerceptronBackPropagation(self.epocas,self.alpha,self.neuronios_por_camada)
                    .sigmoide(entrada)
                else: 
                    ycalculado=entrada2.dot(self.pesos[i])
                    yestimado=MultiLayerPerceptronBackPropagation(self.epocas,self.alpha,self.neuronios_por_camada)
                    .sigmoide(ycalculado)
                    previsoes.append(np.round(yestimado)[0])
        return(np.array(previsoes))
                    
    def acuracia(self,a,b):
        cont=0
        for i in range(len(b)):
            if a[i]==b[i]:
                cont+=1
        return(np.array([np.round(cont/len(b),3)])[0])
    #Função sigmoide
    def sigmoide(self,x):
        return(1/(1+np.exp(-x)))
    #Derivada da função sigmoide. 
    def dsigmoide(self,x):
        return(np.exp(-x)/(1+np.exp(-x))**2)  

# Partição dos dados em treino e teste estratificado pelas Classes.

In [35]:
xtreino,xteste,ytreino,yteste=train_test_split(x,y,test_size=0.3,stratify=y)

# Exemplos de modelos 

# Exemplo 1, rede com camada oculta de 4 neurônios, com 30 épocas e taxa de aprendizagem de 0.01

In [36]:
modelo=MultiLayerPerceptronBackPropagation(30,0.01,np.array([4,1]))

In [37]:
modelo.fit(xtreino,ytreino)

Função custo é:  132.0 na 1 época
Função custo é:  102.0 na 2 época
Função custo é:  83.0 na 3 época
Função custo é:  59.0 na 4 época
Função custo é:  48.0 na 5 época
Função custo é:  54.0 na 6 época
Função custo é:  53.0 na 7 época
Função custo é:  55.0 na 8 época
Função custo é:  57.0 na 9 época
Função custo é:  56.0 na 10 época
Função custo é:  56.0 na 11 época
Função custo é:  56.0 na 12 época
Função custo é:  55.0 na 13 época
Função custo é:  55.0 na 14 época
Função custo é:  55.0 na 15 época
Função custo é:  54.0 na 16 época
Função custo é:  55.0 na 17 época
Função custo é:  54.0 na 18 época
Função custo é:  54.0 na 19 época
Função custo é:  55.0 na 20 época
Função custo é:  55.0 na 21 época
Função custo é:  55.0 na 22 época
Função custo é:  54.0 na 23 época
Função custo é:  54.0 na 24 época
Função custo é:  54.0 na 25 época
Função custo é:  54.0 na 26 época
Função custo é:  54.0 na 27 época
Função custo é:  54.0 na 28 época
Função custo é:  54.0 na 29 época
Função custo é:  54.0

In [38]:
modelo.predict(xteste)

array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1.])

In [39]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.651

# Exemplo 2, rede com camada oculta de 3 neurônios com 100 épocas e taxa de aprendizagem de 0.001

In [40]:
modelo=MultiLayerPerceptronBackPropagation(100,0.001,np.array([3,1]))

In [41]:
modelo.fit(xtreino,ytreino)

Função custo é:  141.0 na 1 época
Função custo é:  141.0 na 2 época
Função custo é:  141.0 na 3 época
Função custo é:  141.0 na 4 época
Função custo é:  141.0 na 5 época
Função custo é:  141.0 na 6 época
Função custo é:  141.0 na 7 época
Função custo é:  141.0 na 8 época
Função custo é:  141.0 na 9 época
Função custo é:  141.0 na 10 época
Função custo é:  141.0 na 11 época
Função custo é:  141.0 na 12 época
Função custo é:  141.0 na 13 época
Função custo é:  141.0 na 14 época
Função custo é:  141.0 na 15 época
Função custo é:  141.0 na 16 época
Função custo é:  141.0 na 17 época
Função custo é:  141.0 na 18 época
Função custo é:  141.0 na 19 época
Função custo é:  141.0 na 20 época
Função custo é:  141.0 na 21 época
Função custo é:  141.0 na 22 época
Função custo é:  141.0 na 23 época
Função custo é:  141.0 na 24 época
Função custo é:  141.0 na 25 época
Função custo é:  141.0 na 26 época
Função custo é:  141.0 na 27 época
Função custo é:  141.0 na 28 época
Função custo é:  141.0 na 29 

In [42]:
modelo.predict(xteste)

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       0.])

In [43]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.663

# Exemplo 3, rede com camada oculta de 5 neurônios com 200 épocas e taxa de aprendizagem de 0.001

In [44]:
modelo=MultiLayerPerceptronBackPropagation(200,0.001,np.array([5,1]))

In [45]:
modelo.fit(xtreino,ytreino)

Função custo é:  111.0 na 1 época
Função custo é:  106.0 na 2 época
Função custo é:  105.0 na 3 época
Função custo é:  106.0 na 4 época
Função custo é:  106.0 na 5 época
Função custo é:  105.0 na 6 época
Função custo é:  102.0 na 7 época
Função custo é:  99.0 na 8 época
Função custo é:  98.0 na 9 época
Função custo é:  97.0 na 10 época
Função custo é:  94.0 na 11 época
Função custo é:  91.0 na 12 época
Função custo é:  91.0 na 13 época
Função custo é:  89.0 na 14 época
Função custo é:  87.0 na 15 época
Função custo é:  87.0 na 16 época
Função custo é:  88.0 na 17 época
Função custo é:  86.0 na 18 época
Função custo é:  86.0 na 19 época
Função custo é:  81.0 na 20 época
Função custo é:  81.0 na 21 época
Função custo é:  81.0 na 22 época
Função custo é:  79.0 na 23 época
Função custo é:  77.0 na 24 época
Função custo é:  79.0 na 25 época
Função custo é:  78.0 na 26 época
Função custo é:  76.0 na 27 época
Função custo é:  74.0 na 28 época
Função custo é:  69.0 na 29 época
Função custo é: 

In [46]:
modelo.predict(xteste)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [47]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.686

# Exemplo 4, rede com camada oculta de 3 neurônios com 30 épocas e taxa de aprendizagem de 0.1

In [55]:
modelo=MultiLayerPerceptronBackPropagation(30,0.1,np.array([3,1]))

In [56]:
modelo.fit(xtreino,ytreino)

Função custo é:  136.0 na 1 época
Função custo é:  60.0 na 2 época
Função custo é:  58.0 na 3 época
Função custo é:  57.0 na 4 época
Função custo é:  58.0 na 5 época
Função custo é:  57.0 na 6 época
Função custo é:  56.0 na 7 época
Função custo é:  56.0 na 8 época
Função custo é:  55.0 na 9 época
Função custo é:  54.0 na 10 época
Função custo é:  55.0 na 11 época
Função custo é:  56.0 na 12 época
Função custo é:  56.0 na 13 época
Função custo é:  57.0 na 14 época
Função custo é:  58.0 na 15 época
Função custo é:  57.0 na 16 época
Função custo é:  56.0 na 17 época
Função custo é:  54.0 na 18 época
Função custo é:  54.0 na 19 época
Função custo é:  53.0 na 20 época
Função custo é:  52.0 na 21 época
Função custo é:  47.0 na 22 época
Função custo é:  46.0 na 23 época
Função custo é:  46.0 na 24 época
Função custo é:  44.0 na 25 época
Função custo é:  43.0 na 26 época
Função custo é:  43.0 na 27 época
Função custo é:  43.0 na 28 época
Função custo é:  42.0 na 29 época
Função custo é:  41.0 

In [57]:
modelo.predict(xteste)

array([1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1.])

In [58]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.663

# Exemplo 5, rede com 150 épocas, taxa de aprendizagem 0.01 e 4 neurônios na camada oculta.

In [72]:
modelo=MultiLayerPerceptronBackPropagation(150,0.01,np.array([3,1]))

In [73]:
modelo.fit(xtreino,ytreino)

Função custo é:  141.0 na 1 época
Função custo é:  142.0 na 2 época
Função custo é:  137.0 na 3 época
Função custo é:  87.0 na 4 época
Função custo é:  59.0 na 5 época
Função custo é:  59.0 na 6 época
Função custo é:  59.0 na 7 época
Função custo é:  59.0 na 8 época
Função custo é:  59.0 na 9 época
Função custo é:  59.0 na 10 época
Função custo é:  59.0 na 11 época
Função custo é:  59.0 na 12 época
Função custo é:  59.0 na 13 época
Função custo é:  59.0 na 14 época
Função custo é:  59.0 na 15 época
Função custo é:  59.0 na 16 época
Função custo é:  59.0 na 17 época
Função custo é:  59.0 na 18 época
Função custo é:  59.0 na 19 época
Função custo é:  59.0 na 20 época
Função custo é:  59.0 na 21 época
Função custo é:  59.0 na 22 época
Função custo é:  59.0 na 23 época
Função custo é:  59.0 na 24 época
Função custo é:  59.0 na 25 época
Função custo é:  59.0 na 26 época
Função custo é:  59.0 na 27 época
Função custo é:  59.0 na 28 época
Função custo é:  59.0 na 29 época
Função custo é:  59.

In [74]:
modelo.predict(xteste)

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [75]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.709